In [ ]:
import os
import sys
import json
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import cross_val_score, LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from tqdm import tqdm

warnings.filterwarnings('ignore')

# add project to path
project_root = Path('/Volumes/usb drive/pd-interpretability')
sys.path.insert(0, str(project_root))

from src.data import ItalianPVSDataset, MDVRKCLDataset, ArkansasDataset
from src.features import extract_clinical_features

print(f"pytorch version: {torch.__version__}")
print(f"device: {torch.device('cpu')}")

In [ ]:
# configuration
config = {
    'data_root': project_root / 'data',
    'results_root': project_root / 'results',
    'output_dir': project_root / 'results' / 'probing',
    'model_path': project_root / 'results' / 'final_model',
    'datasets': ['italian_pvs'],  # start with Italian PVS
    'n_layers': 12,  # wav2vec2-base has 12 layers
    'hidden_size': 768,
    'random_seed': 42,
    'n_permutations': 1000,  # for significance testing
    'cv_method': 'logo',  # leave-one-group-out (subject-wise)
}

# create output directory
config['output_dir'].mkdir(parents=True, exist_ok=True)

# set random seeds
np.random.seed(config['random_seed'])
torch.manual_seed(config['random_seed'])

print(f"configuration:")
for k, v in config.items():
    if not isinstance(v, Path):
markdown
markdown
## 2. Layer-Wise PD Classification Probing

For each layer, fit a logistic regression probe to predict PD/HC from pooled activations. Use leave-one-subject-out cross-validation for medical rigor. Report mean accuracy, F1, and AUC for each layer.
code
python
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

logo = LeaveOneGroupOut()
scaler = StandardScaler()

layerwise_results = {}

for layer_idx in range(n_layers):
    X = activations_by_layer[layer_idx]
    X_scaled = scaler.fit_transform(X)
    y = np.array(labels)
    groups = np.array(subject_ids)

    fold_accuracies = []
    fold_f1s = []
    fold_aucs = []

    for train_idx, test_idx in logo.split(X_scaled, y, groups=groups):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        probe = LogisticRegression(max_iter=1000, random_state=42, solver='lbfgs')
        probe.fit(X_train, y_train)
        y_pred = probe.predict(X_test)
        y_proba = probe.predict_proba(X_test)[:, 1]

        fold_accuracies.append(accuracy_score(y_test, y_pred))
        fold_f1s.append(f1_score(y_test, y_pred, zero_division=0))
        if len(np.unique(y_test)) > 1:
            fold_aucs.append(roc_auc_score(y_test, y_proba))

    layerwise_results[layer_idx] = {
        'accuracy_mean': np.mean(fold_accuracies),
        'accuracy_std': np.std(fold_accuracies),
        'f1_mean': np.mean(fold_f1s),
        'f1_std': np.std(fold_f1s),
        'auc_mean': np.mean(fold_aucs) if fold_aucs else np.nan,
        'auc_std': np.std(fold_aucs) if fold_aucs else np.nan
    }

print('Layer-wise PD classification probing complete.')
for layer_idx in range(n_layers):
    result = layerwise_results[layer_idx]
    print(f'Layer {layer_idx:2d}: acc={result[accuracy_mean]:.3f}±{result[accuracy_std]:.3f}, f1={result[f1_mean]:.3f}±{result[f1_std]:.3f}, auc={result[auc_mean]:.3f}±{result[auc_std]:.3f}')
        print(f"  {k}: {v}")

## 1. Load Dataset and Extract Activations
markdown
markdown
## 1. Activation Loading and Preprocessing

This section loads the Italian PVS dataset and pre-extracted activations from the fine-tuned Wav2Vec2 model (Phase 04).
All preprocessing steps are documented for reproducibility. LOSO splits are used for subject-wise rigor.
code
python
import os
import sys
import json
import numpy as np
import pandas as pd
from pathlib import Path

project_root = Path('/Volumes/usb drive/pd-interpretability')
sys.path.insert(0, str(project_root))

# Load dataset metadata
from src.data import ItalianPVSDataset

data_root = project_root / 'data'
activations_path = project_root / 'data' / 'activations' / 'activations'

# Load subject and label info
dataset = ItalianPVSDataset(root_dir=data_root / 'raw' / 'italian_pvs', max_duration=10.0, target_sr=16000)
subject_ids = [s['subject_id'] for s in dataset.samples]
labels = [s['label'] for s in dataset.samples]

print(f'Dataset loaded: {len(dataset)} samples')
print(f'Subjects: {len(set(subject_ids))}')
print(f'Label distribution: {pd.Series(labels).value_counts().to_dict()}')
code
python
# Load activations from Phase 04
n_layers = 12
hidden_size = 768
activations_by_layer = {i: [] for i in range(n_layers)}

for layer_idx in range(n_layers):
    layer_file = activations_path / f'layer_{layer_idx}.npy'
    if layer_file.exists():
        activations_by_layer[layer_idx] = np.load(layer_file)
        print(f'Loaded activations for layer {layer_idx}: {activations_by_layer[layer_idx].shape}')
    else:
        print(f'Warning: missing activations for layer {layer_idx}')
code
python
# Sanity check: all layers should have activations for all samples
for layer_idx in range(n_layers):
    acts = activations_by_layer[layer_idx]
    assert acts.shape[0] == len(dataset), f'Layer {layer_idx} activation count mismatch'
    assert acts.shape[1] == hidden_size, f'Layer {layer_idx} hidden size mismatch'
print('All activations loaded and validated.')

In [ ]:
# load Italian PVS dataset
dataset = ItalianPVSDataset(
    root_dir=config['data_root'] / 'raw' / 'italian_pvs',
    task=None,  # all tasks
    max_duration=10.0,
    target_sr=16000
)

print(f"dataset size: {len(dataset)} samples")
print(f"subject groups: {dataset.get_subject_groups()}")

# get subject labels for cross-validation
sample_subjects = [dataset.samples[i]['subject_id'] for i in range(len(dataset))]
sample_labels = [dataset.samples[i]['label'] for i in range(len(dataset))]

print(f"\nlabel distribution:")
unique, counts = np.unique(sample_labels, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  class {label}: {count} samples")

In [ ]:
# load pre-trained model for activation extraction
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor

device = 'cpu'

# try to load fine-tuned model, fall back to base model
if config['model_path'].exists():
    print(f"loading fine-tuned model from {config['model_path']}")
    model = Wav2Vec2ForSequenceClassification.from_pretrained(str(config['model_path']))
else:
    print("fine-tuned model not found, loading base model")
    model = Wav2Vec2ForSequenceClassification.from_pretrained(
        'facebook/wav2vec2-base-960h',
        num_labels=2
    )

model = model.to(device)
model.eval()

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('facebook/wav2vec2-base-960h')

print(f"model architecture:")
print(f"  n_layers: {len(model.wav2vec2.encoder.layers)}")
print(f"  hidden_size: {model.wav2vec2.config.hidden_size}")

In [ ]:
# extract activations from all layers
print("extracting activations from all samples and layers...")

activations_by_layer = {i: [] for i in range(config['n_layers'])}
labels_list = []
subject_ids_list = []

with torch.no_grad():
    for idx in tqdm(range(len(dataset)), desc="extracting activations"):
        sample = dataset[idx]
        audio = sample['input_values'].to(device)
        label = sample['label']
        subject_id = sample['subject_id']
        
        # forward pass with hidden states
        outputs = model.wav2vec2(
            audio.unsqueeze(0),
            output_hidden_states=True
        )
        
        hidden_states = outputs.hidden_states  # tuple of (batch, seq_len, hidden_size)
        
        # extract and pool each layer's activation
        for layer_idx, hidden in enumerate(hidden_states[1:]):  # skip input layer
            # mean pooling over sequence dimension
            pooled = hidden.mean(dim=1).squeeze(0).cpu().numpy()
            activations_by_layer[layer_idx].append(pooled)
        
        labels_list.append(label)
        subject_ids_list.append(subject_id)

# convert to arrays
for layer_idx in range(config['n_layers']):
    activations_by_layer[layer_idx] = np.array(activations_by_layer[layer_idx])

labels = np.array(labels_list)
subject_ids = np.array(subject_ids_list)

print(f"\nactivation shapes:")
for i in range(min(3, config['n_layers'])):
    print(f"  layer {i}: {activations_by_layer[i].shape}")
print(f"  ...")
print(f"labels shape: {labels.shape}")
print(f"subject_ids shape: {subject_ids.shape}")

## 2. Load Clinical Features

In [ ]:
# extract or load clinical features
clinical_features_path = config['data_root'] / 'clinical_features' / 'italian_pvs_features.csv'

if clinical_features_path.exists():
    print("loading pre-extracted clinical features...")
    clinical_df = pd.read_csv(clinical_features_path)
else:
    print("extracting clinical features from audio...")
    clinical_features_list = []
    
    for idx in tqdm(range(len(dataset)), desc="extracting clinical features"):
        sample = dataset[idx]
        audio_path = sample.get('path')
        
        if audio_path and Path(audio_path).exists():
            try:
                features = extract_clinical_features(audio_path)
                features['subject_id'] = sample['subject_id']
                features['label'] = sample['label']
                features['path'] = audio_path
                clinical_features_list.append(features)
            except Exception as e:
                print(f"  failed on {audio_path}: {e}")
    
    clinical_df = pd.DataFrame(clinical_features_list)
    clinical_df.to_csv(clinical_features_path, index=False)
    print(f"saved clinical features to {clinical_features_path}")

print(f"\nclinical features shape: {clinical_df.shape}")
print(f"columns: {clinical_df.columns.tolist()}")
print(f"\nfeature summary:")
print(clinical_df.describe())

## 3. Layer-Wise PD Classification Probing

In [ ]:
# run layer-wise PD classification probing
print("running layer-wise PD classification probing...\n")

layerwise_results = {}
scaler = StandardScaler()
logo = LeaveOneGroupOut()

for layer_idx in tqdm(range(config['n_layers']), desc="layer-wise probing"):
    X = activations_by_layer[layer_idx]
    X_scaled = scaler.fit_transform(X)
    
    # logistic regression probe
    probe = LogisticRegression(max_iter=1000, random_state=config['random_seed'], solver='lbfgs')
    
    # leave-one-subject-out cross-validation
    predictions = np.zeros_like(labels, dtype=float)
    fold_accuracies = []
    fold_f1s = []
    fold_aucs = []
    
    for train_idx, test_idx in logo.split(X, labels, groups=subject_ids):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = labels[train_idx], labels[test_idx]
        
        probe.fit(X_train, y_train)
        y_pred = probe.predict(X_test)
        y_proba = probe.predict_proba(X_test)[:, 1]
        
        predictions[test_idx] = y_proba
        
        fold_accuracies.append(accuracy_score(y_test, y_pred))
        fold_f1s.append(f1_score(y_test, y_pred, zero_division=0))
        
        if len(np.unique(y_test)) > 1:
            fold_aucs.append(roc_auc_score(y_test, y_proba))
    
    layerwise_results[layer_idx] = {
        'accuracy_mean': np.mean(fold_accuracies),
        'accuracy_std': np.std(fold_accuracies),
        'accuracy_folds': fold_accuracies,
        'f1_mean': np.mean(fold_f1s),
        'f1_std': np.std(fold_f1s),
        'auc_mean': np.mean(fold_aucs) if fold_aucs else np.nan,
        'auc_std': np.std(fold_aucs) if fold_aucs else np.nan,
    }

print("\nlayer-wise PD classification probing results:")
print("-" * 60)
for layer_idx in range(config['n_layers']):
    result = layerwise_results[layer_idx]
    print(f"layer {layer_idx:2d}: acc={result['accuracy_mean']:.3f}±{result['accuracy_std']:.3f}, "
          f"f1={result['f1_mean']:.3f}±{result['f1_std']:.3f}, "
          f"auc={result['auc_mean']:.3f}±{result['auc_std']:.3f}")

In [ ]:
# identify layers with significant discriminative power
accuracies = [layerwise_results[i]['accuracy_mean'] for i in range(config['n_layers'])]
accuracy_stds = [layerwise_results[i]['accuracy_std'] for i in range(config['n_layers'])]

# threshold: accuracy > 0.65 (better than chance for binary classification)
important_layers = [i for i, acc in enumerate(accuracies) if acc > 0.65]

print(f"\nlayers with significant PD discrimination (acc > 0.65): {important_layers}")
print(f"peak accuracy: {max(accuracies):.3f} at layer {np.argmax(accuracies)}")

In [ ]:
# visualize layer-wise probing
fig, ax = plt.subplots(figsize=(12, 6))

layers = np.arange(config['n_layers'])
ax.errorbar(layers, accuracies, yerr=accuracy_stds, fmt='o-', capsize=5, 
            color='steelblue', ecolor='darkgray', linewidth=2, markersize=8, label='probing accuracy')

# chance level
ax.axhline(y=0.5, color='red', linestyle='--', linewidth=2, alpha=0.7, label='chance (0.5)')
ax.axhline(y=0.65, color='orange', linestyle=':', linewidth=2, alpha=0.7, label='significance threshold (0.65)')

# highlight important layers
for layer in important_layers:
    ax.axvline(x=layer, alpha=0.2, color='green')

ax.set_xlabel('Layer', fontsize=12, fontweight='bold')
ax.set_ylabel('Accuracy (Leave-One-Subject-Out CV)', fontsize=12, fontweight='bold')
ax.set_title('Layer-Wise PD Classification Probing\nWav2Vec2-Base (12 layers)', fontsize=14, fontweight='bold')
ax.set_xticks(layers)
ax.set_ylim([0.4, 1.0])
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11, loc='lower right')

plt.tight_layout()
plt.savefig(config['output_dir'] / 'fig_p5_01_layerwise_probing.png', dpi=300, bbox_inches='tight')
plt.show()

print("saved figure: fig_p5_01_layerwise_probing.png")

## 4. Clinical Feature Probing

In [ ]:
# select clinical features to probe
clinical_feature_cols = ['jitter_local', 'shimmer_local', 'hnr', 'f0_mean_std']

# check which features are available
available_features = [col for col in clinical_feature_cols if col in clinical_df.columns]
print(f"available clinical features: {available_features}")

# if not enough features, use what's available
if len(available_features) < len(clinical_feature_cols):
    print(f"note: using available features only")
    clinical_feature_cols = available_features[:4]  # use first 4 available

print(f"\nprobing for: {clinical_feature_cols}")

In [ ]:
# clinical feature probing: predict each feature from layer activations
print("running clinical feature probing...\n")

clinical_probing_results = {feature: {} for feature in clinical_feature_cols}
scaler = StandardScaler()
logo = LeaveOneGroupOut()

for feature_name in tqdm(clinical_feature_cols, desc="clinical features"):
    # get feature values
    feature_values = clinical_df[feature_name].values
    
    # binarize at median for classification (high vs. normal)
    feature_median = np.median(feature_values)
    feature_binary = (feature_values > feature_median).astype(int)
    
    for layer_idx in range(config['n_layers']):
        X = activations_by_layer[layer_idx]
        X_scaled = scaler.fit_transform(X)
        
        # ridge regression for continuous prediction
        ridge = Ridge(alpha=1.0)
        
        # cross-validation
        cv_scores = cross_val_score(ridge, X_scaled, feature_values, cv=logo, 
                                     groups=subject_ids, scoring='r2')
        
        # also compute binary classification accuracy
        probe = LogisticRegression(max_iter=1000, random_state=config['random_seed'])
        binary_scores = cross_val_score(probe, X_scaled, feature_binary, cv=logo,
                                        groups=subject_ids, scoring='accuracy')
        
        clinical_probing_results[feature_name][layer_idx] = {
            'r2_mean': np.mean(cv_scores),
            'r2_std': np.std(cv_scores),
            'binary_acc_mean': np.mean(binary_scores),
            'binary_acc_std': np.std(binary_scores),
        }

print("\nclinical feature probing results:")
print("-" * 80)
for feature_name in clinical_feature_cols:
    print(f"\n{feature_name.upper()}:")
    best_layer = max(range(config['n_layers']), 
                     key=lambda i: clinical_probing_results[feature_name][i]['r2_mean'])
    best_r2 = clinical_probing_results[feature_name][best_layer]['r2_mean']
    print(f"  best layer: {best_layer}, r²={best_r2:.3f}")

In [ ]:
# create clinical feature encoding heatmap
heatmap_data = np.zeros((len(clinical_feature_cols), config['n_layers']))

for i, feature_name in enumerate(clinical_feature_cols):
    for layer_idx in range(config['n_layers']):
        heatmap_data[i, layer_idx] = clinical_probing_results[feature_name][layer_idx]['r2_mean']

fig, ax = plt.subplots(figsize=(14, 5))

im = ax.imshow(heatmap_data, cmap='RdYlGn', aspect='auto', vmin=0, vmax=max(0.3, heatmap_data.max()))

ax.set_xlabel('Layer', fontsize=12, fontweight='bold')
ax.set_ylabel('Clinical Feature', fontsize=12, fontweight='bold')
ax.set_title('Clinical Feature Encoding Across Layers\n(Ridge Regression R² scores)', fontsize=14, fontweight='bold')

ax.set_xticks(range(config['n_layers']))
ax.set_yticks(range(len(clinical_feature_cols)))
ax.set_yticklabels(clinical_feature_cols)

# add values to heatmap
for i in range(len(clinical_feature_cols)):
    for j in range(config['n_layers']):
        text = ax.text(j, i, f'{heatmap_data[i, j]:.2f}',
                       ha="center", va="center", color="black", fontsize=8)

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('R² Score', fontsize=11)

plt.tight_layout()
plt.savefig(config['output_dir'] / 'fig_p5_02_clinical_feature_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("saved figure: fig_p5_02_clinical_feature_heatmap.png")

## 5. Statistical Validation and Selectivity Scoring

In [ ]:
# control task: probe for random labels (should be at chance)
print("running control task: probing random labels...\n")

random_probe_results = {}
scaler = StandardScaler()

for layer_idx in tqdm(range(config['n_layers']), desc="control probing"):
    X = activations_by_layer[layer_idx]
    X_scaled = scaler.fit_transform(X)
    
    # random labels
    random_labels = np.random.RandomState(config['random_seed']).randint(0, 2, len(labels))
    
    probe = LogisticRegression(max_iter=1000, random_state=config['random_seed'])
    
    random_scores = cross_val_score(probe, X_scaled, random_labels, cv=logo,
                                     groups=subject_ids, scoring='accuracy')
    
    random_probe_results[layer_idx] = {
        'accuracy_mean': np.mean(random_scores),
        'accuracy_std': np.std(random_scores),
    }

print("\ncontrol probing results (random labels):")
print("-" * 60)
for layer_idx in range(config['n_layers']):
    result = random_probe_results[layer_idx]
    print(f"layer {layer_idx:2d}: acc={result['accuracy_mean']:.3f}±{result['accuracy_std']:.3f}")

In [ ]:
# compute selectivity scores
selectivity_scores = {}

for layer_idx in range(config['n_layers']):
    pd_acc = layerwise_results[layer_idx]['accuracy_mean']
    control_acc = random_probe_results[layer_idx]['accuracy_mean']
    
    # selectivity = (target_acc - control_acc) / control_acc
    selectivity = (pd_acc - control_acc) / max(control_acc, 0.01)
    selectivity_scores[layer_idx] = selectivity

print("\nselectivity scores (PD vs. random):")
print("-" * 60)
for layer_idx in range(config['n_layers']):
    sel = selectivity_scores[layer_idx]
    print(f"layer {layer_idx:2d}: selectivity = {sel:.3f}")

# layers with selectivity > 0.2 are considered selective
selective_layers = [i for i, sel in selectivity_scores.items() if sel > 0.2]
print(f"\nselective layers (selectivity > 0.2): {selective_layers}")

## 6. Save Results and Summary

In [ ]:
# compile all results
full_results = {
    'config': {
        'n_layers': config['n_layers'],
        'dataset': 'italian_pvs',
        'n_samples': len(labels),
        'cv_method': config['cv_method'],
    },
    'layerwise_pd_probing': layerwise_results,
    'clinical_feature_probing': clinical_probing_results,
    'control_probing': random_probe_results,
    'selectivity_scores': selectivity_scores,
    'important_layers': important_layers,
    'selective_layers': selective_layers,
}

# save to json
results_path = config['output_dir'] / 'probing_results.json'
with open(results_path, 'w') as f:
    json.dump(full_results, f, indent=2, default=str)

print(f"saved results to {results_path}")

In [ ]:
# summary statistics
print("\n" + "="*80)
print("PHASE 05 PROBING EXPERIMENTS SUMMARY")
print("="*80)

print(f"\nDATASET:")
print(f"  samples: {len(labels)}")
print(f"  subjects: {len(np.unique(subject_ids))}")
print(f"  pd/hc: {np.sum(labels==1)}/{np.sum(labels==0)}")

print(f"\nLAYER-WISE PD CLASSIFICATION PROBING:")
print(f"  best layer: {np.argmax(accuracies)}")
print(f"  peak accuracy: {max(accuracies):.3f}")
print(f"  significant layers (acc > 0.65): {important_layers}")
print(f"  selective layers (sel > 0.2): {selective_layers}")

print(f"\nCLINICAL FEATURE ENCODING:")
for feature_name in clinical_feature_cols:
    best_layer = max(range(config['n_layers']), 
                     key=lambda i: clinical_probing_results[feature_name][i]['r2_mean'])
    best_r2 = clinical_probing_results[feature_name][best_layer]['r2_mean']
    print(f"  {feature_name}: layer {best_layer}, r²={best_r2:.3f}")

print(f"\nCONTROL VALIDATION:")
control_accs = [random_probe_results[i]['accuracy_mean'] for i in range(config['n_layers'])]
print(f"  random label accuracy: {np.mean(control_accs):.3f} ± {np.std(control_accs):.3f}")
print(f"  (should be near 0.5 for binary classification)")

print("\n" + "="*80)
print("ABSTRACT READY: PRELIMINARY RESULTS CONFIRMED")
print("="*80)

In [ ]:
# generate source data for figures
source_data = {
    'layerwise_probing': {
        'layers': list(range(config['n_layers'])),
        'accuracy': accuracies,
        'accuracy_std': accuracy_stds,
    },
    'clinical_features_heatmap': heatmap_data.tolist(),
    'clinical_feature_names': clinical_feature_cols,
}

source_data_path = config['output_dir'] / 'p5_source_data.json'
with open(source_data_path, 'w') as f:
    json.dump(source_data, f, indent=2)

print(f"saved source data to {source_data_path}")